In [92]:
# 라이브러리 임포트
import pandas as pd
import numpy as np

# 베르누이 나이브베이즈를 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB

# 모델의 정확도 평가를 위해 임포트
from sklearn.metrics import accuracy_score
from sklearn import metrics

# 결과값을 일치시키기 위해 랜덤값 고정
np.random.seed(5)

## 문제정의
- 베르누이 나이브베이지 분류모델을 사용하여 스팸 메일을 분류

## 데이터 수집
- 이번 실습에서는 간단한 스팸 메일 분류 실습을 위해 아래 이메일 타이틀과 스팸여부가 있는 데이터를 로드

In [93]:
email_list = [
                {'email title': 'free game only today', 'spam': True},
                {'email title': 'cheapest flight deal', 'spam': True},
                {'email title': 'limited time offer only today only today', 'spam': True},
                {'email title': 'today meeting schedule', 'spam': False},
                {'email title': 'your flight schedule attached', 'spam': False},
                {'email title': 'your credit card statement', 'spam': False}
             ]

In [94]:
df = pd.DataFrame(email_list)
df.head()

,email title,spam
0,free game only today,True
1,cheapest flight deal,True
2,limited time offer only today only today,True
3,today meeting schedule,False
4,your flight schedule attached,False


## 데이터 전처리 
- sklearn의 베르누이 나이브베이즈 분류기는 숫자만을 다루기 때문에, True와 False를 1과 0으로 치환합니다. 

In [95]:
df['label'] = df['spam'].map({True : 1 , False : 0})
df

,email title,spam,label
0,free game only today,True,1
1,cheapest flight deal,True,1
2,limited time offer only today only today,True,1
3,today meeting schedule,False,0
4,your flight schedule attached,False,0
5,your credit card statement,False,0


In [96]:
# 학습에 사용될 데이터와 분류값을 구분해준다. 
df_x = df['email title']
df_y = df['label']

베르누이 나이브베이즈의 입력 데이터는 고정된 크기의 벡터로써, 0과 1로 구분된 데이터이여야 합니다.  
sklearn의 CountVectorizer를 사용하여 쉽게 구현할 수 있습니다.  
CountVectorizer는 입력된 데이터(6개의 이메일)에 출현된 모든 단어의 갯수만큼의 크기의 벡터를 만든 후,  
각각의 이메일을 그 고정된 벡터로 표현합니다.  
binary=True를 파라미터를 넘겨줌으로써, 각각의 이메일마다 단어가 한번 이상 출현하면 1, 출현하지 않을 경우 0으로 표시하게 합니다.

In [97]:
cv  = CountVectorizer(binary=True)
x_traincv = cv.fit_transform(df_x) # 알고리즘 내부에서 분산 밎 표준편차등을 새로!! 계산해서 담아줌.
x_traincv #객체를 반환함으로 주소 값만 보임.

<6x17 sparse matrix of type '<class 'numpy.int64'>'
	with 23 stored elements in Compressed Sparse Row format>

In [98]:
encoded_input = x_traincv.toarray()
encoded_input

array([[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]], dtype=int64)

In [99]:
cv.inverse_transform(encoded_input[[0]])

[array(['free', 'game', 'only', 'today'], dtype='<U9')]

In [100]:
cv.get_feature_names() # 순서 확인

C:\ProgramData\Anaconda3\envs\tf_cpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['attached',
 'card',
 'cheapest',
 'credit',
 'deal',
 'flight',
 'free',
 'game',
 'limited',
 'meeting',
 'offer',
 'only',
 'schedule',
 'statement',
 'time',
 'today',
 'your']

## 베르누이 나이브베이즈 분류 모델 생성
- 스펨 메일 분류
- BernoulliNB 기본적으로 스무딩을 지원하므로, 학습데이터에 없는 단어가 테스트에 출현해도 분류를 이상없이 진행합니다.

In [101]:
# 학습데이터로 베르누이 분류기를 학습
bnb = BernoulliNB()
y_train = df_y.astype('int')
bnb.fit(x_traincv, y_train)

BernoulliNB()

In [102]:
test_email_list = [
                {'email title': 'free flight offer', 'spam': True},
                {'email title': 'hey traveler free flight deal', 'spam': True},
                {'email title': 'limited free game offer', 'spam': True},
                {'email title': 'today flight schedule', 'spam': False},
                {'email title': 'your credit card attached', 'spam': False},
                {'email title': 'free credit card offer only today', 'spam': False}
             ]

In [103]:
test_df = pd.DataFrame(test_email_list)

test_df['label'] = test_df['spam'].map({True:1, False:0})

test_x = test_df['email title']
test_y = test_df['label']

x_testcv = cv.transform(test_x) # 알고리즘 내부에서 분산 밎 표준편차등을 계산한 것(cv)을 변환시킬뿐

In [104]:
# 테스트 데이터로 모델을 테스트 
pred = bnb.predict(x_testcv)

print(accuracy_score(test_y, pred))

0.8333333333333334
